# How to save parameters along with the function 

This is a workaround for the problem of how to save *and restore* the parameters of a workflow (e.g., sigma for blurring, threshold values, etc) along with the functions. The idea is simple:

Instead of setting workflow steps with, for instance, `w.set('blurring', blur_function, 'input', sigma=5)`, this notebook creates a partial function of `blur_function()` which is then added to the workflow. When the workflow is loaded and the partial function is passed to magicgui, the parameters are restored correctly.

In [1]:
from napari_workflows import Workflow
from napari.types import ImageData
import napari
import yaml
from skimage.filters import threshold_otsu, gaussian

import inspect
import magicgui
from functools import partial
from napari_tools_menu import make_gui

### First fix:

Turn function into partial object before putting it into the workflow

In [12]:
filename = r'C:/Users/johan/Desktop/blobs.tif'

In [ ]:
# define segmentation
def threshold(image: ImageData, value: float = 128) -> ImageData:
    return image > value

def add(imageA: ImageData, imageB:ImageData) -> ImageData:
    return imageA + imageB

def blur(image: ImageData, sigma: float=2) -> ImageData:
    return gaussian(image, sigma=sigma)

Create a workflow from the above frunctions

In [29]:
kwargs = {'sigma': 5}

_blur = partial(blur, **kwargs)

w = Workflow()
w.set('blurring', _blur, 'input', **kwargs)
w.set('binarization', threshold, 'blurring')

Save and reload workflow

In [31]:
with open('workflow.yaml', 'w') as f:
    yaml.dump(w, f)
    
with open("workflow.yaml", "rb") as stream:
    w=yaml.unsafe_load(stream)
    
w._tasks

{'binarization': (<function __main__.threshold(image: <function NewType.<locals>.new_type at 0x0000017C68F5F9D0>, value: float = 1) -> <function NewType.<locals>.new_type at 0x0000017C68F5F9D0>>,
  'blurring',
  1),
 'blurring': (functools.partial(<function blur at 0x0000017C132B79D0>, sigma=5),
  'input',
  5)}

Result: The created widgets are executable and the parameters a re correctly passed to the passed function (which is a bit surprising as partial functions have the preset parameters hardcoded in their scope...)

In [35]:
viewer = napari.Viewer()
viewer.open(filename)

steps = w._tasks.keys()
for key in w._tasks.keys():
    func = w._tasks[key][0]
    widget = magicgui.magicgui(func)
    viewer.window.add_dock_widget(widget)

### Second fix:
Turn function into partial object after loading it

In [ ]:
w = Workflow()
w.set('blurring', blur, 'input', **kwargs)
w.set('binarization', threshold, 'blurring')

In [37]:
with open('_workflow.yaml', 'w') as f:
    yaml.dump(w, f)
    
with open("_workflow.yaml", "rb") as stream:
    w=yaml.unsafe_load(stream)
    
w._tasks

{'binarization': (<function __main__.threshold(image: <function NewType.<locals>.new_type at 0x0000017C68F5F9D0>, value: float = 1) -> <function NewType.<locals>.new_type at 0x0000017C68F5F9D0>>,
  'blurring',
  1),
 'blurring': (<function __main__.blur(image: <function NewType.<locals>.new_type at 0x0000017C68F5F9D0>, sigma: float = 2) -> <function NewType.<locals>.new_type at 0x0000017C68F5F9D0>>,
  'input',
  5)}

Similar solution as first fix, but partialising the function is done after restoring it from the workflow file and before passing it to magicgui.

In [40]:
viewer = napari.Viewer()
viewer.open(filename)

for task in w._tasks.keys():
    func = w._tasks[task][0]  # extract function from workflow
    keyword_list = list(inspect.signature(func).parameters.keys())  # get name of kwargs
    
    # We need to retrieve a dictionary with parameter: value from the saved function
    kw_dict = {}
    for kw, val in zip(keyword_list, w._tasks[task][1:]):
        kw_dict[kw] = val
    kw_dict.pop('image')
    
    func = partial(w._tasks[task][0], **kw_dict)
    
    widget = magicgui.magicgui(func, auto_call=True)
    viewer.window.add_dock_widget(widget)

### Third fix:
Get function from workflow file and change its default parameters settings by changing `func.__defaults__`:

In [41]:
viewer = napari.Viewer()
viewer.open(filename)

[<Image layer 'blobs' at 0x17c14e4ab80>]

In [42]:
for task in w._tasks.keys():
    func = w._tasks[task][0]  # extract function from workflow
    keyword_list = list(inspect.signature(func).parameters.keys())  # get name of kwargs
    
    # retrieve keyword - saved value dictionary from saved workflow
    kw_dict = {}
    for kw, val in zip(keyword_list, w._tasks[task][1:]):
        kw_dict[kw] = val
    kw_dict.pop('image')
    
    # change defaults of restored function and add to viewer
    defaults = ()
    for key in kw_dict.keys():
        defaults += tuple([kw_dict[key]])
    
    func.__defaults__ = defaults
    widget = make_gui(func, viewer)
    
    viewer.window.add_dock_widget(widget)    

In [22]:
func

functools.partial(<function blur at 0x0000017C6BA120D0>, sigma=5)